In [2]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

**Problem definition**

In [6]:
PROBLEM_SIZE = 10
NUM_SETS = 20
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

def distance(state):
    return PROBLEM_SIZE - sum(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))


**Simple Solution**

In [7]:
frontier = SimpleQueue()
frontier.put(State(set(), set(range(NUM_SETS))))

counter = 0
current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken - {action})
        frontier.put(new_state)
    current_state = frontier.get()

print(f"Solved in {counter:,} steps")
print(current_state)

Solved in 2,268 steps
State(taken={9, 5, 15}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19})


**Improved solution**

In [11]:
def key(state):
    indices = state.taken
    if len(indices) != 0:
        return -sum([sum(SETS[i]) for i in indices])
    else:
        return 0

chosen_key = distance

frontier = PriorityQueue()
init = State(set(), set(range(NUM_SETS)))
frontier.put((chosen_key(init), init))

counter = 0
k, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken - {action})
        frontier.put((chosen_key(new_state),new_state))
    k, current_state = frontier.get()


print(f"Solved in {counter:,} steps")
print(current_state)

Solved in 3 steps
State(taken={9, 19, 15}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18})


A* finds solution in an optimal way = expand minimum number of nodes
- Complete and Optimally efficient
f(n) = g(n) + h(n)

h(.) is admissable (not overestimate the cost)

{1, 2, 3, 4}